# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [1]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [2]:
# Load dataset
data_311 = pd.read_csv(os.getenv('DFDATA') + '/erm2-nwe9/1473456214/erm2-nwe9.csv', usecols=['Created Date', 'Complaint Type',
                        'Descriptor', 'Community Board', 'Borough'])
data_311.shape

(13588781, 5)

In [3]:
data_311['Community Board'].head()

0    Unspecified BROOKLYN
1            11 MANHATTAN
2            11 MANHATTAN
3               01 QUEENS
4             15 BROOKLYN
Name: Community Board, dtype: object

In [4]:
data_community = pd.read_csv('https://data.cityofnewyork.us/resource/w3c6-35wg.csv')

In [5]:
data_community['JURISDICTION NAME'].unique()

array(['Bronx CD 001', 'Bronx CD 002', 'Bronx CD 003', 'Bronx CD 004',
       'Bronx CD 005', 'Bronx CD 006', 'Bronx CD 007', 'Bronx CD 008',
       'Bronx CD 009', 'Bronx CD 010', 'Bronx CD 011', 'Bronx CD 012',
       'Brooklyn CD 001', 'Brooklyn CD 002', 'Brooklyn CD 003',
       'Brooklyn CD 004', 'Brooklyn CD 005', 'Brooklyn CD 006',
       'Brooklyn CD 007', 'Brooklyn CD 008', 'Brooklyn CD 009',
       'Brooklyn CD 010', 'Brooklyn CD 011', 'Brooklyn CD 012',
       'Brooklyn CD 013', 'Brooklyn CD 014', 'Brooklyn CD 015',
       'Brooklyn CD 016', 'Brooklyn CD 017', 'Brooklyn CD 018',
       'Manhattan CD 001', 'Manhattan CD 002', 'Manhattan CD 003',
       'Manhattan CD 004', 'Manhattan CD 005', 'Manhattan CD 006',
       'Manhattan CD 007', 'Manhattan CD 008', 'Manhattan CD 009',
       'Manhattan CD 010', 'Manhattan CD 011', 'Manhattan CD 012',
       'Queens CD 001', 'Queens CD 002', 'Queens CD 003', 'Queens CD 004',
       'Queens CD 005', 'Queens CD 006', 'Queens CD 007', 'Q

In [6]:
len(data_311['Community Board'].unique())

77

# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [7]:
# Check if all Boroughs and Community Districts are represented in the Data 

In [8]:
# How many unique values do we have? 
len(data_311['Community Board'].unique())

77

In [9]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts. 
data_311['Community Board'].unique()

array(['Unspecified BROOKLYN', '11 MANHATTAN', '01 QUEENS', '15 BROOKLYN',
       '06 MANHATTAN', '01 STATEN ISLAND', '11 QUEENS', '03 BROOKLYN',
       '03 MANHATTAN', '01 MANHATTAN', 'Unspecified QUEENS',
       '10 MANHATTAN', '0 Unspecified', 'Unspecified STATEN ISLAND',
       'Unspecified MANHATTAN', '09 QUEENS', '08 QUEENS',
       'Unspecified BRONX', '02 BRONX', '01 BROOKLYN', '12 BRONX',
       '10 QUEENS', '08 BRONX', '04 BRONX', '12 BROOKLYN', '12 MANHATTAN',
       '07 MANHATTAN', '04 BROOKLYN', '08 BROOKLYN', '05 BRONX',
       '09 BRONX', '10 BROOKLYN', '03 STATEN ISLAND', '05 BROOKLYN',
       '02 MANHATTAN', '01 BRONX', '17 BROOKLYN', '02 STATEN ISLAND',
       '09 MANHATTAN', '03 QUEENS', '06 BRONX', '07 BROOKLYN',
       '14 BROOKLYN', '07 BRONX', '18 BROOKLYN', '14 QUEENS', '06 QUEENS',
       '11 BROOKLYN', '06 BROOKLYN', '13 BROOKLYN', '11 BRONX',
       '08 MANHATTAN', '09 BROOKLYN', '16 BROOKLYN', '04 QUEENS',
       '12 QUEENS', '07 QUEENS', '10 BRONX', '05 MAN

In [10]:
# Check for duplicates? Are these plausible?

In [11]:
real_districts = data_community['JURISDICTION NAME'].unique()

In [12]:
# splits the district so they will be in the form borough number
comm_dist = []
for district in real_districts:
    split = district.split(' ')
    comm_dist.append(split[0] + ' ' + split[2][1:3])

In [13]:
# changes the district name so that borough is written in short form per the last question
boroughs = [['Bronx ', 'BX'], ['Brooklyn ', 'BK'], ['Manhattan ', 'MN'], ['Queens ', 'QN'], ['Richmond ', 'RI']]
for br in boroughs:
    comm_dist = [s.replace(br[0], br[1]) for s in comm_dist]

In [14]:
# changes case on community districts so that the boroughs code will work
data_311['Community District'] = map(lambda x: x.title(), data_311['Community Board'])

In [15]:
dist_311 = list(data_311['Community District'])

In [16]:
dist_corrected = []
for district in dist_311:
    split = district.split(' ')
    dist_corrected.append(split[1] + ' ' + split[0])


In [17]:
boroughs = [['Bronx ', 'BX'], ['Brooklyn ', 'BK'], ['Manhattan ', 'MN'], ['Queens ', 'QN'], ['Staten ', 'RI']]
for br in boroughs:
    dist_corrected = [s.replace(br[0], br[1]) for s in dist_corrected]

In [18]:
data_311['Community District'] = dist_corrected

In [19]:
# What about missing values? Can you detect any patterns? 

In [20]:
data_311[data_311['Complaint Type'] == '']

,Created Date,Complaint Type,Descriptor,Community Board,Borough,Community District


In [21]:
data_311[data_311['Descriptor'] == '']

,Created Date,Complaint Type,Descriptor,Community Board,Borough,Community District


In [22]:
# there don't seem to be any major problems in this cut of the dataset, but for the bad district names

In [23]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?

In [24]:
# Generate marker for unplausible Community Districts
# How do these districts look like?  # contain words like unspecified 

In [25]:
data_311['marker'] = [1 if x in comm_dist else 0 for x in data_311['Community District']]

In [26]:
data_311[data_311.marker == 0].head(10)

,Created Date,Complaint Type,Descriptor,Community Board,Borough,Community District,marker
0,02/02/2015 02:15:00 PM,Water Conservation,Water Meter Broken/Leaking - Private Residence...,Unspecified BROOKLYN,BROOKLYN,BKUnspecified,0
7,02/02/2015 05:12:00 PM,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),Unspecified BROOKLYN,BROOKLYN,BKUnspecified,0
12,02/02/2015 07:57:00 AM,Water System,Hydrant Defective (WC2),Unspecified QUEENS,QUEENS,QNUnspecified,0
16,02/02/2015 01:58:01 PM,DOF Literature Request,Request to Update Property Description for Tax...,0 Unspecified,Unspecified,Unspecified 0,0
17,02/02/2015 01:37:00 PM,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),Unspecified BROOKLYN,BROOKLYN,BKUnspecified,0
18,02/02/2015 12:47:00 PM,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),Unspecified STATEN ISLAND,STATEN ISLAND,RIUnspecified,0
19,02/02/2015 11:37:00 AM,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),Unspecified MANHATTAN,MANHATTAN,MNUnspecified,0
20,02/02/2015 11:01:00 AM,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),Unspecified QUEENS,QUEENS,QNUnspecified,0
21,02/02/2015 10:02:00 AM,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),Unspecified QUEENS,QUEENS,QNUnspecified,0
22,02/02/2015 09:59:00 AM,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),Unspecified STATEN ISLAND,STATEN ISLAND,RIUnspecified,0


In [27]:
# Drop the marked districts

In [28]:
df_311 = data_311[data_311.marker > 0]

In [29]:
df_311.head()

,Created Date,Complaint Type,Descriptor,Community Board,Borough,Community District,marker
1,02/02/2015 02:16:04 AM,Noise - Commercial,Loud Music/Party,11 MANHATTAN,MANHATTAN,MN11,1
2,02/02/2015 02:17:59 AM,Noise - Commercial,Loud Music/Party,11 MANHATTAN,MANHATTAN,MN11,1
3,02/02/2015 05:11:34 PM,Illegal Parking,Blocked Hydrant,01 QUEENS,QUEENS,QN01,1
4,02/02/2015 01:06:13 PM,Illegal Parking,Blocked Hydrant,15 BROOKLYN,BROOKLYN,BK15,1
5,02/02/2015 04:43:29 PM,Illegal Parking,Blocked Hydrant,06 MANHATTAN,MANHATTAN,MN06,1


In [30]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level

reduced_311 = df_311.groupby('Community District').count()

In [31]:
# Safe reduced data frame (Community District level)

In [32]:
reduced_311.head()

,Created Date,Complaint Type,Descriptor,Community Board,Borough,marker
Community District,,,,,,
BK01,243046,243046,241260,243046,243046,243046
BK02,157589,157589,155880,157589,157589,157589
BK03,268591,268591,266104,268591,268591,268591
BK04,185247,185247,184010,185247,185247,185247
BK05,251986,251986,249944,251986,251986,251986


In [33]:
counts = pd.DataFrame(reduced_311.marker)

# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [34]:
# Population by Community District
df_pop = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')

In [35]:
# Check variables in file
df_pop.columns

Index([u'FIPS', u'cd_id', u'Total Population',
       u'Population Density (per sq. mile)', u'% Total Population: Male',
       u'% Total Population: 18 to 24 Years',
       u'% Total Population: 25 to 34 Years',
       u'% Total Population: 35 to 44 Years',
       u'% Population 5 Years And Over: Speak Only English',
       u'% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       u'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       u'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       u'% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       u'% Employed Civilian Population 16 Years And Over: Service occupations',
       u'% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       

In [36]:
# How many community districts are in file? 
len(df_pop.cd_id.unique())

59

In [37]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 

#Older people, richer people, whiter gentrifiers who don't understand the neighborhood they moved into and don't care about their 
# neighbors' preferences so call to complain rather than having a conversation with them

In [38]:
# Save data frame

In [39]:
merged = df_pop.merge(counts, left_on = 'cd_id', right_index = True)
merged.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations",marker
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20,123832
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02,227016
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42,122448
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20,166939
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04,104502


In [40]:
# Infrastructure by Community District
df_infr = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv')
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [41]:
# Check variables in file
len(df_infr['Qualifying Name'].unique())

55

In [42]:
df_infr['Qualifying Name'].unique()

array(['NYC-Bronx Community District 8--Riverdale, New York',
       'NYC-Bronx Community District 12--Wakefield, New York',
       'NYC-Bronx Community District 10--Co-op City, New York',
       'NYC-Bronx Community District 11--Pelham Parkway, New York',
       'NYC-Bronx Community District 3 & 6--Belmont, New York',
       'NYC-Bronx Community District 7--Bedford Park, New York',
       'NYC-Bronx Community District 5--Morris Heights, New York',
       'NYC-Bronx Community District 4--Concourse, New York',
       'NYC-Bronx Community District 9--Castle Hill, New York',
       'NYC-Bronx Community District 1 & 2--Hunts Point, New York',
       'NYC-Manhattan Community District 12--Washington Heights, New York',
       'NYC-Manhattan Community District 9--Hamilton Heights, New York',
       'NYC-Manhattan Community District 10--Central Harlem PUMA, New York',
       'NYC-Manhattan Community District 11--East Harlem PUMA, New York',
       'NYC-Manhattan Community District 8--Upper Eas

In [43]:
# How many community districts are in file? 
# 55 unique entries, but some entries are combined and comprise multple districts

In [44]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
df_infr.columns

Index([u'FIPS', u'Geographic Identifier', u'Qualifying Name', u'Households',
       u'Households: With An Internet Subscription',
       u'Households: Dial-Up Alone', u'Households: Dsl',
       u'Households: With Mobile Broadband',
       u'Households: Without Mobile Broadband', u'Households: Cable Modem',
       u'Households: With Mobile Broadband.1',
       u'Households: Without Mobile Broadband.1', u'Households: Fiber-Optic',
       u'Households: With Mobile Broadband.2',
       u'Households: Without Mobile Broadband.2',
       u'Households: Satellite Internet Service',
       u'Households: With Mobile Broadband.3',
       u'Households: Without Mobile Broadband.3',
       u'Households: Two or More Fixed Broadband Types, or Other',
       u'Households: With Mobile Broadband.4',
       u'Households: Without Mobile Broadband.4',
       u'Households: Mobile Broadband Alone or With Dialup',
       u'Households: Internet Access Without A Subscription',
       u'Households: No Internet Acc

In [80]:
# df of all households with broadband alone or with dialup
base_df = pd.concat([df_infr['Qualifying Name'], df_infr['Households']], axis = 1)

In [82]:
mobile_bb = df_infr['Households: With Mobile Broadband']
for x in range(1, 5):
    mobile_bb = mobile_bb + df_infr['Households: With Mobile Broadband.' + str(x)]

In [85]:
mobile_bb_df = pd.concat([base_df, pd.DataFrame(mobile_bb)], axis = 1)

In [88]:
mobile_bb_df.columns = ['CD', 'Households', 'Mobile_BB_Subscribers']

In [89]:
mobile_bb_df.head()

,CD,Households,Mobile_BB_Subscribers
0,"NYC-Bronx Community District 8--Riverdale, New...",42035,15359
1,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,10486
2,"NYC-Bronx Community District 10--Co-op City, N...",47050,11425
3,NYC-Bronx Community District 11--Pelham Parkwa...,44922,10748
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,14907


In [46]:
# Aggregate internet type by high and low connections

In [141]:
# created a dataframe with an indicator for whether percentage of HH with internet subscription was more or less than half the mea
# as this seems most worthwhile for later analysis 
internet_df = pd.concat([base_df, df_infr['% Households: With An Internet Subscription']], axis = 1)

In [145]:
mean_internet = internet_df['% Households: With An Internet Subscription'].mean()

In [ ]:
internet_percent = internet_df['% Households: With An Internet Subscription']

In [146]:
internet_df['hi_lo'] = [1 if x > mean_internet else 0 for x in internet_df['% Households: With An Internet Subscription']]

In [149]:
internet_df.head()

,Qualifying Name,Households,% Households: With An Internet Subscription,hi_lo
0,"NYC-Bronx Community District 8--Riverdale, New...",42035,75.64,1
1,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,71.92,0
2,"NYC-Bronx Community District 10--Co-op City, N...",47050,69.56,0
3,NYC-Bronx Community District 11--Pelham Parkwa...,44922,71.24,0
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,61.68,0


# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [48]:
# Harmonize identifier of dataframe 1- done in first step

In [139]:
reduced_311.head()

,Created Date,Complaint Type,Descriptor,Community Board,Borough,marker
Community District,,,,,,
BK01,243046,243046,241260,243046,243046,243046
BK02,157589,157589,155880,157589,157589,157589
BK03,268591,268591,266104,268591,268591,268591
BK04,185247,185247,184010,185247,185247,185247
BK05,251986,251986,249944,251986,251986,251986


In [49]:
# Harmonize identifier of dataframe 2

In [140]:
merged.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations",marker
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20,123832
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02,227016
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42,122448
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20,166939
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04,104502


In [255]:
# Harmonize identifier of dataframe 3

(59, 4)

In [257]:
internet_df.head(6)

,Qualifying Name,Households,% Households: With An Internet Subscription,hi_lo
0,"NYC-Bronx Community District 8--Riverdale, New...",42035,75.64,1
1,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,71.92,0
2,"NYC-Bronx Community District 10--Co-op City, N...",47050,69.56,0
3,NYC-Bronx Community District 11--Pelham Parkwa...,44922,71.24,0
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,61.68,0
5,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,61.68,0


In [371]:
# regexes out everything but borough name and number and saves as separate list
district = []
numbers = []
for x in range(len(internet_df)): 
    z = re.sub(r'NYC-', '',internet_df['Qualifying Name'][x])
    z = re.sub(r'Community District ', '', z)
    z = re.sub(r'--.*', '', z)
    split = re.split('' '', z)
    district.extend(split)
    numbers.extend(n)

In [372]:
boroughs = [['Bronx', 'BX'], ['Brooklyn', 'BK'], ['Manhattan', 'MN'], ['Queens', 'QN'], ['Staten Island', 'RI']]
for br in boroughs:
    
    district = [s.replace(br[0], br[1]) for s in district]

In [374]:
# puts the districts into common form 
for x in range(len(district)):
    if re.search('&', district[x]) == None:
        if int(district[x][3:]) < 10:
            district[x] = district[x][0:2] + '0' + district[x][3:]
        else: 
            district[x] = re.sub(' ', '', district[x])

In [376]:
internet_df['district'] = district

In [51]:
# Link the 3 dataframes

In [393]:
# dropped the data for which we did not have internet data because the internet study covered multiple districts 
linked = merged.merge(internet_df, left_on = 'cd_id', right_on = 'district', how ='inner')

In [52]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than ot

In [396]:
# divides complaints by population for better comparison between districts
linked['per capita complaints'] = linked.marker / linked ['Total Population']

In [402]:
linked[linked.hi_lo == 1.0]['per capita complaints'].mean() > linked[linked.hi_lo == 0.0]['per capita complaints'].mean()

False

There are actually more per capita complaints in areas with less internet access

are there more complaints per capita in areas where there is an economically diverse population or a cohesive population?

In [506]:
linked['income_std'] = linked.loc[: , '% Households: Less than $10,000':'% Households: $200,000 or More'].std(axis = 1)

In [509]:
avg_std_dev = linked['income_std'].mean()

In [513]:
linked[linked.income_std > avg_std_dev]['per capita complaints'].mean() > linked[linked.income_std < avg_std_dev]['per capita complaints'].mean()

False

There are more per capita complaints in areas with less economic diversity as approximated by standad deviation of the percents falling into each income category. This is not especially robust as income categories are not of an equal size in dollar values.

In [514]:
avg_complaints = linked['per capita complaints'].mean()

In [516]:
# shows means for all variables for districts with fewer complaints than avg
linked[linked['per capita complaints'] < avg_complaints].mean()

FIPS                                                                                                                                                              3.603986e+06
Total Population                                                                                                                                                  1.593531e+05
Population Density (per sq. mile)                                                                                                                                 3.723533e+04
% Total Population: Male                                                                                                                                          4.773250e+01
% Total Population: 18 to 24 Years                                                                                                                                8.442500e+00
% Total Population: 25 to 34 Years                                                                                           

In [460]:
# shows means for all variables for districts with more complaints than avg

In [517]:
linked[linked['marker'] > avg_complaints].mean()

FIPS                                                                                                                                                              3.603956e+06
Total Population                                                                                                                                                  1.540286e+05
Population Density (per sq. mile)                                                                                                                                 4.532426e+04
% Total Population: Male                                                                                                                                          4.750745e+01
% Total Population: 18 to 24 Years                                                                                                                                9.552553e+00
% Total Population: 25 to 34 Years                                                                                           

More dense areas feature a greater number of per capita complaints. Once that is accounted for, I actually found that there were more complaints in lower income areas, perhaps because they are often more dense and there is more to complain about. There were also more per capita complaints in areas with more people age 18-24, whether because they complain more or because more people complain aobut them. 